In [ ]:
!pip install -r requirements.txt


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=da4b3c167a2aa7243d361053d7ec62fa64bd35467fa8f40daeea6c45ddc4702e
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
import re
import torch
from PIL import Image
import fitz  # PyMuPDF
import io
from difflib import SequenceMatcher
import docx
import os
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import bert_score
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import warnings
warnings.filterwarnings('ignore')

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# Load TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed").to(device)


✅ Using device: cpu


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Define OCR function**

In [ ]:
def ocr_pdf_trocr(pdf_path, first_page=None, last_page=None, dpi=300):
    """Convert PDF to images using PyMuPDF and perform OCR with TrOCR"""
    doc = fitz.open(pdf_path)
    start_page = 0 if first_page is None else first_page - 1
    end_page = len(doc) if last_page is None else last_page
    page_texts = []

    for page_num in range(start_page, end_page):
        page = doc.load_page(page_num)
        mat = fitz.Matrix(dpi / 72, dpi / 72)
        pix = page.get_pixmap(matrix=mat)
        img_data = pix.tobytes("ppm")
        image = Image.open(io.BytesIO(img_data)).convert("RGB")

        pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        generated_ids = model.generate(pixel_values)
        txt = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        page_texts.append(txt)
        print(f"[debug] OCR’d Page {page_num + 1}: {len(txt)} chars")

    doc.close()
    return page_texts


# **Define helper functions (text reading + preprocessing)**

In [ ]:
def read_docx(file_path):
    """Read text from a Word document"""
    doc = docx.Document(file_path)
    return "\n".join(p.text for p in doc.paragraphs)

def preprocess_text(text):
    """Clean and normalize text for fair comparison"""
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s.,;:!?()\-]", "", text)
    return text.strip()


# **Define evaluation metric functions (CER, WER, BLEU, ROUGE, BERTScore)**

In [ ]:
# Character Error Rate (CER)
def calculate_character_error_rate(reference, hypothesis):
    ref, hyp = list(reference), list(hypothesis)
    d = np.zeros((len(ref) + 1, len(hyp) + 1))
    for i in range(len(ref) + 1): d[i, 0] = i
    for j in range(len(hyp) + 1): d[0, j] = j

    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            cost = 0 if ref[i-1] == hyp[j-1] else 1
            d[i, j] = min(d[i-1, j] + 1, d[i, j-1] + 1, d[i-1, j-1] + cost)

    cer = d[len(ref), len(hyp)] / len(ref) * 100 if ref else 0
    return cer

# Word Error Rate (WER)
def calculate_word_error_rate(reference, hypothesis):
    ref, hyp = reference.split(), hypothesis.split()
    d = np.zeros((len(ref) + 1, len(hyp) + 1))
    for i in range(len(ref) + 1): d[i, 0] = i
    for j in range(len(hyp) + 1): d[0, j] = j

    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            cost = 0 if ref[i-1] == hyp[j-1] else 1
            d[i, j] = min(d[i-1, j] + 1, d[i, j-1] + 1, d[i-1, j-1] + cost)

    wer = d[len(ref), len(hyp)] / len(ref) * 100 if ref else 0
    return wer

# BLEU
def calculate_bleu_score(reference, hypothesis):
    ref_words, hyp_words = reference.split(), hypothesis.split()
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_words], hyp_words,
                         weights=(0.25, 0.25, 0.25, 0.25),
                         smoothing_function=smoothing) * 100

# ROUGE
def calculate_rouge_scores(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return {k: v.fmeasure * 100 for k, v in scores.items()}

# BERTScore
def calculate_bertscore(reference, hypothesis):
    P, R, F1 = bert_score.score([hypothesis], [reference], lang="en", verbose=False)
    return F1.item() * 100


# **Word difference highlighter**

In [ ]:
def highlight_word_differences(reference, hypothesis):
    ref_words = reference.split()
    hyp_words = hypothesis.split()
    matcher = SequenceMatcher(None, ref_words, hyp_words)
    result = []

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":
            result.extend(ref_words[i1:i2])
        elif tag == "replace":
            result.append(f"[SUB: {' '.join(ref_words[i1:i2])} -> {' '.join(hyp_words[j1:j2])}]")
        elif tag == "delete":
            result.append(f"[DEL: {' '.join(ref_words[i1:i2])}]")
        elif tag == "insert":
            result.append(f"[INS: {' '.join(hyp_words[j1:j2])}]")
    return " ".join(result)


# **Main execution block**

In [ ]:
# ===== MAIN EXECUTION =====

pdf_file = "a01-011u_scannedpaper.pdf"
gt_file = "a01-011u_groundtruth.docx"

print(f"OCR on {pdf_file}...")
ocr_pages = ocr_pdf_trocr(pdf_file)
ground_truth = read_docx(gt_file)

# ---- Save OCR text output ----
ocr_output_file = "a01-011u_trocr_output.txt"
with open(ocr_output_file, "w", encoding="utf-8") as f:
    f.write("\n\n".join(ocr_pages))

print(f"✅ OCR complete and saved as '{ocr_output_file}'.")

# ---- Metrics ----
gt_clean = preprocess_text(ground_truth)
ocr_clean = preprocess_text(" ".join(ocr_pages))

cer = calculate_character_error_rate(gt_clean, ocr_clean)
wer = calculate_word_error_rate(gt_clean, ocr_clean)
bleu = calculate_bleu_score(gt_clean, ocr_clean)
rouge = calculate_rouge_scores(gt_clean, ocr_clean)
bertscore_val = calculate_bertscore(gt_clean, ocr_clean)

# ---- Display metrics ----
print("\n📊 EVALUATION METRICS")
print(f"CER: {cer:.2f}%")
print(f"WER: {wer:.2f}%")
print(f"BLEU: {bleu:.2f}%")
print(f"ROUGE-1: {rouge['rouge1']:.2f}%")
print(f"ROUGE-2: {rouge['rouge2']:.2f}%")
print(f"ROUGE-L: {rouge['rougeL']:.2f}%")
print(f"BERTScore: {bertscore_val:.2f}%")

# ---- Show sample differences ----
print("\n🔍 WORD-LEVEL DIFFERENCES:")
print(highlight_word_differences(gt_clean, ocr_clean)[:1000])

# ---- Generate and save comprehensive report ----
report_filename = "trocr_comprehensive_report.txt"

report_content = f"""
==================== TrOCR Comprehensive Report ====================

📄 PDF File: {pdf_file}
🧾 Ground Truth File: {gt_file}
🗂️ OCR Output File: {ocr_output_file}

==================== METRIC SCORES ====================
Character Error Rate (CER): {cer:.2f} %
Word Error Rate (WER): {wer:.2f} %
BLEU Error: {bleu:.2f} %
ROUGE-1: {rouge['rouge1']:.2f} %
ROUGE-2: {rouge['rouge2']:.2f} %
ROUGE-L: {rouge['rougeL']:.2f} %
BERTScore Error: {bertscore_val:.2f} %

==================== NOTES ====================
- Model: TrOCR (Microsoft)
- Evaluation performed on aligned scanned paper dataset
- Generated automatically in Colab environment
====================================================================
"""

with open(report_filename, "w", encoding="utf-8") as f:
    f.write(report_content)

print(f"\n✅ Comprehensive report saved as '{report_filename}'")


OCR on a01-011u_scannedpaper.pdf...
[debug] OCR’d Page 1: 4 chars
✅ OCR complete and saved as 'a01-011u_trocr_output.txt'.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📊 EVALUATION METRICS
CER: 98.96%
WER: 100.00%
BLEU: 0.00%
ROUGE-1: 0.00%
ROUGE-2: 0.00%
ROUGE-L: 0.00%
BERTScore: 78.21%

🔍 WORD-LEVEL DIFFERENCES:
[SUB: delegates from mr. kenneth kaundas united national independence party (280,000 members) and mr. harry nkumbulas african national congress (400,000) will meet in london today to discuss a common course of action. sir roy is violently opposed to africans getting an elected majority in northern rhodesia, but the colonial secretary, mr. lain macleod, is insisting on a policy of change. -> item]

✅ Comprehensive report saved as 'trocr_comprehensive_report.txt'
